In [2]:
cd /home/sam/bf-nn

/home/sam/bf-nn


/home/sam/anaconda3/envs/py10-coreset/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from src.models import *
from src.dataset import * 
import numpy as np
import torch

In [29]:
def three_node_path(edge_val, beta = 100):
    x = torch.tensor([[0], [edge_val], [beta]], dtype=torch.float)
    edge_index = torch.tensor([[0, 1, 1, 2, 0, 1, 2], 
                               [1, 0, 2, 1, 0, 1, 2]])
    edge_attr = torch.unsqueeze(torch.tensor([edge_val, edge_val, 0, 0, 0, 0, 0], dtype=torch.float), dim=1)
    y = torch.tensor([[0], [edge_val], [edge_val]], dtype=torch.float)
    return x, edge_index, edge_attr, y

In [35]:
# load final model

bf_nn_final = SimpleBFLayer()

model_pth = '/data/sam/bf-nn/l1-regularized-loss-2/final_model.pt'

gnn_state = torch.load(model_pth, map_location='cpu')

bf_nn_final.load_state_dict(gnn_state)

<All keys matched successfully>

In [36]:
w2 = bf_nn_final.w_2.detach().numpy()
W1 = bf_nn_final.W_1.weight.detach().numpy()

b1 = bf_nn_final.W_1.bias.detach().numpy()
b2 = bf_nn_final.b_2.detach().numpy()

In [37]:
print(W1, w2, b1, b2)
print(w2 * W1)
print(w2 * b1 + b2)

[[0.04008514 0.9963649 ]] [1.0026937] [2.9952236e-05] [7.181458e-06]
[[0.04019311 0.99904877]]
[3.7214377e-05]


In [38]:
def create_star(num_nodes, beta=200):
    x = [0]
    edge_index = [[], []]
    edge_attr = []

    for i in range(1, num_nodes):
        x.append(beta)
        edge_index[0].append(0)
        edge_index[1].append(i)
        edge_index[0].append(i)
        edge_index[1].append(0)

        edge_attr.append(i)
        edge_attr.append(i)
    
    # add self loops
    self_loops =  torch.vstack((torch.arange(start = 0, end = num_nodes), torch.arange(start = 0, end=num_nodes)))
    edge_index = torch.hstack((torch.tensor(edge_index), self_loops))
    edge_attr = torch.cat((torch.tensor(edge_attr, dtype=torch.float), torch.zeros(num_nodes)))
    x =torch.unsqueeze(torch.tensor(x, dtype=torch.float), dim=1)
    edge_attr = torch.unsqueeze(edge_attr, dim=1)
    return x, edge_index, edge_attr

In [39]:
x, edge_index, edge_attr = create_star(11, beta=100)

In [40]:
x, edge_index, edge_attr, _ = three_node_path(1, beta=200)

In [41]:
bf_nn_final(x, edge_index, edge_attr)

tensor([[3.7214e-05],
        [4.0230e-02],
        [4.0230e-02]], grad_fn=<ReluBackward0>)